In [1]:
import pandas as pd
akshit_df = './mlchallenge_set_2021.tsv'
akshit_valid = './mlchallenge_set_validation.tsv'
sam_df = './drive/MyDrive/mlchallenge_set_2021_edited.txt'
#sam_valid = './mlchallenge_set_validation.tsv'
#SA_valid=pd.read_table('/Users/shivankagrawal/Documents/ebay/mlchallenge_set_validation.tsv',header=None)
#SA_df=pd.read_table('/Users/shivankagrawal/Documents/ebay/mlchallenge_set_2021.tsv',header=None)
#df=SA_df
#valid=SA_valid
df = pd.read_table(sam_df)
#valid = pd.read_table(sam_valid,sep='\t')
df.columns=['category','primary_image_url','All Links','Tags','index']
#valid.columns=['ID', 'Group']

In [2]:
import re
from collections import Counter
freq=Counter()
attribute=[['']]*len(df)
trialrange=3000
for x in range(trialrange):#range(int(len(df)/10)):#len(df)
    attribute[x]=df.iloc[x,3].lower()
    attribute[x] = re.sub(r'[()]','', attribute[x])
    attribute[x] = re.split(r',', attribute[x])
    attribute[x] = [a.split(':') for a in attribute[x]]
    freq+=Counter([i[0] for i in attribute[x]])
    tempdict={}
    for i in attribute[x]:

            try:
                tempdict[i[0]]=float(i[1])
            except:
                try:
                    tempdict[i[0]]=i[1]
                except:
                    pass
    attribute[x]=tempdict

df['seg']=attribute
#print(df['seg'])

In [ ]:
for i in df['seg'][0:10000]:
  if 'color' in i:
    print(i['color'])

In [ ]:
Brands=[]
Images=[]
Colors = []
color_images = []
simple_colors = {'black','white','red','blue','green','yellow','brown','purple','pink','gray','grey'}
print(trialrange)
for i in range(trialrange):
    try:
        #df['seg'].iloc[i]['brand']
        #df['primary_image_url'].iloc[i]
        #print(df['seg'].iloc[i]['brand'])
        #print(df['primary_image_url'].iloc[i])
        #if df['seg'].iloc[i]['brand'] == 'nike' or df['seg'].iloc[i]['brand'] == 'adidas':
        #   Brands.append(df['seg'].iloc[i]['brand'])
        #    Images.append(df['primary_image_url'].iloc[i])
        for key in simple_colors:
          if key in df['seg'].iloc[i]['color']:
              Colors.append(key)
              color_images.append(df['primary_image_url'].iloc[i])
              continue
    except:
        continue
        #Brands.remove[-1]
        #print('not possible at: ',i)
print(color_images)

3000
['https://i.ebayimg.com/00/s/MTA1OFgxMTM0/z/KPIAAOSw4A5YoCoa/$_1.JPG?set_id=880000500F', 'https://i.ebayimg.com/00/s/MTA1OFgxMTM0/z/KPIAAOSw4A5YoCoa/$_1.JPG?set_id=880000500F', 'https://i.ebayimg.com/00/s/MTIwMFgxNjAw/z/flIAAOSwxH1UCdW5/$_1.JPG?set_id=880000500F', 'http://i.ebayimg.com/00/s/ODAwWDEwNjc=/z/XHcAAOSwY45UUagU/$_1.JPG?set_id=880000500F', 'http://i.ebayimg.com/00/s/MTYwMFgxMjAw/z/u~cAAOSwu4BVzRvK/$_1.JPG?set_id=880000500F', 'http://i.ebayimg.com/00/s/MTIwMFgxNjAw/z/~ZMAAOSw~OVWz8~R/$_1.JPG?set_id=880000500F', 'http://i.ebayimg.com/00/s/MTIwMFgxNjAw/z/~ZMAAOSw~OVWz8~R/$_1.JPG?set_id=880000500F', 'https://i.ebayimg.com/00/s/MTAzN1gxMzQ4/z/qPYAAOSwzvlW-~xs/$_1.JPG?set_id=880000500F', 'http://i.ebayimg.com/00/s/MTYwMFgxMjAw/z/J5gAAOSw~oFXHQaN/$_1.JPG?set_id=880000500F', 'http://i.ebayimg.com/00/s/NzM2WDczNg==/z/1k0AAOxyBotTYcP9/$_57.JPG?set_id=880000500F', 'http://i.ebayimg.com/00/s/ODAwWDEwNjc=/z/nR0AAOSw8gVX3s1l/$_57.JPG?set_id=8800005007', 'http://i.ebayimg.com/00/s/ODAw

In [ ]:
n = 500
from PIL import Image, ImageFile
import requests
from io import BytesIO
import numpy as np
ImageFile.LOAD_TRUNCATED_IMAGES = True

image_array = []
images = []
max_height = 0
max_width = 0
i = 0
for img in color_images[0:n]:
    response = requests.get(img)
    if i%200 == 0:
        print(i)
    i+=1
    img = Image.open(BytesIO(response.content))
    if np.asarray(img).shape[1] > max_width:
        max_width = np.asarray(img).shape[1]
    if np.asarray(img).shape[0] > max_height:
        max_height = np.asarray(img).shape[0]
    images.append(img)
i=0
for img in images:
    if i%200 == 0:
        print(i)
    i+=1
    try:
        img = img.convert('RGB')
        margin = Image.new(img.mode, (max_width, max_height), (64,64,64))
    except:
        img = img.convert('RGB')
        margin = Image.new(img.mode,(max_width,max_height),(64,64,64))
    margin.paste(img, (0, 0))
    image_array.append(np.asarray(margin))

In [ ]:
#ignore this
n = 500
from PIL import Image, ImageFile
import requests
import urllib.request
from io import BytesIO
import numpy as np
import grequests
ImageFile.LOAD_TRUNCATED_IMAGES = True
image_array = []
images = []
max_height = 0
max_width = 0
i = 0
urls = []
for img in color_images[0:n]:
    #response = requests.get(img)
    urls.append(urllib.request.urlretrieve(img))
    #urls.append(img)
print(urls)
glist = (grequests.get(u) for u in urls)
rs = grequests.map(glist)
print(rs)
for g in rs:
    if i%200 == 0:
        print(i)
    i+=1
    img = Image.open(BytesIO(g.content))
    if np.asarray(img).shape[1] > max_width:
        max_width = np.asarray(img).shape[1]
    if np.asarray(img).shape[0] > max_height:
        max_height = np.asarray(img).shape[0]
    images.append(img)



i=0
#code to make images all same size: right now uses max_width and max_height
for img in images:
    if i%200 == 0:
        print(i)
    i+=1
    try:
        img = img.convert('RGB')
        margin = Image.new(img.mode, (max_width, max_height), (64,64,64))
    except:
        img = img.convert('RGB')
        margin = Image.new(img.mode,(max_width,max_height),(64,64,64))
    margin.paste(img, (0, 0))
    image_array.append(np.asarray(margin))


[('/tmp/tmpq7jnyonr', <http.client.HTTPMessage object at 0x7fd57fada9e8>), ('/tmp/tmpni5foy_r', <http.client.HTTPMessage object at 0x7fd57ec86d68>), ('/tmp/tmpknt9rqjg', <http.client.HTTPMessage object at 0x7fd584a85128>), ('/tmp/tmp35jbx56f', <http.client.HTTPMessage object at 0x7fd57f52b860>), ('/tmp/tmpvdrd15xx', <http.client.HTTPMessage object at 0x7fd57ec20828>), ('/tmp/tmpj4fcncbi', <http.client.HTTPMessage object at 0x7fd584b6e0b8>), ('/tmp/tmp3etvqt9s', <http.client.HTTPMessage object at 0x7fd5848f4ba8>), ('/tmp/tmp10qi2dpa', <http.client.HTTPMessage object at 0x7fd57e864e80>), ('/tmp/tmpco9jw2xi', <http.client.HTTPMessage object at 0x7fd584b9d550>), ('/tmp/tmpi9jsjs1w', <http.client.HTTPMessage object at 0x7fd57ec029e8>), ('/tmp/tmp7m5ld69k', <http.client.HTTPMessage object at 0x7fd584b64898>), ('/tmp/tmpbs7e_qar', <http.client.HTTPMessage object at 0x7fd584b9d3c8>), ('/tmp/tmpocf76oeo', <http.client.HTTPMessage object at 0x7fd57ec02da0>), ('/tmp/tmptnr1naxb', <http.client.HTT

AttributeError: ignored

In [ ]:
n = 500
from PIL import Image, ImageFile
import requests
from io import BytesIO
import numpy as np
from concurrent.futures import as_completed
from concurrent.futures import ThreadPoolExecutor
from requests_futures.sessions import FuturesSession

ImageFile.LOAD_TRUNCATED_IMAGES = True
image_array = []
images = []
max_height = 0
max_width = 0
i = 0

f = []
session = FuturesSession(executor=ThreadPoolExecutor(max_workers=10))
for img in color_images[0:n]:
  f.append(session.get(img))
for i in as_completed(f):
  print(i)
  image_array.append(i.result())
print(image_array)

<Future at 0x7fd584c13278 state=finished raised RecursionError>


RecursionError: ignored

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.layers import Dropout

model = models.Sequential()
model.add(layers.Conv2D(4, (3, 3), activation='relu', input_shape=(max_width, max_height, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(4, activation='relu'))
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
brand_dict = {}
num = 0
labels = []
for b in Colors:
    if b not in brand_dict:
        brand_dict[b] = num
        num+=1
    labels.append(brand_dict[b])
print(len(labels))
m = 500
n = round(m*.8)
#train_images = np.asarray(image_array[0:n])
#test_images = np.asarray(image_array[n:m])
#train_labels = np.asarray(labels[0:n])
#test_labels = np.asarray(labels[n:m])
#print(type(test_images[0]))

661


In [ ]:
history = model.fit(np.asarray(image_array[0:500]), np.asarray(labels[0:500]), epochs=30, batch_size = 10,validation_split=.8)

Epoch 1/30
10/10 [==============================] - 15s 646ms/step - loss: 47.6492 - accuracy: 0.4406 - val_loss: 1.3863 - val_accuracy: 0.6209
Epoch 2/30
10/10 [==============================] - 3s 348ms/step - loss: 1.3863 - accuracy: 0.6064 - val_loss: 1.3863 - val_accuracy: 0.6209
Epoch 3/30
10/10 [==============================] - 3s 360ms/step - loss: 1.3863 - accuracy: 0.5801 - val_loss: 1.3863 - val_accuracy: 0.6209
Epoch 4/30
10/10 [==============================] - 3s 349ms/step - loss: 1.3863 - accuracy: 0.7420 - val_loss: 1.3863 - val_accuracy: 0.6209
Epoch 5/30
10/10 [==============================] - 3s 351ms/step - loss: 1.3863 - accuracy: 0.6309 - val_loss: 1.3863 - val_accuracy: 0.6209
Epoch 6/30
10/10 [==============================] - 3s 348ms/step - loss: 1.3863 - accuracy: 0.6918 - val_loss: 1.3863 - val_accuracy: 0.6209
Epoch 7/30
10/10 [==============================] - 3s 353ms/step - loss: 1.3863 - accuracy: 0.6432 - val_loss: 1.3863 - val_accuracy: 0.6209
Epoc